# Random Forest - Temperature Forecast

### Predicting the max temperature for tomorrow in our city using one year of past weather data using Random Forest

Can we predict the tomorrow's maximum temperature for Bangalore, given one year of historical data? 

![title](indiamin_temp.jpg)

#### What data we have? 

- **one year of historical max temperatures data**

**This is a supervised, regression machine learning problem.**

# 1. Load Data

### Columns/Attributes

1) year: 2019 for all data points
2) month: number for month of the year
3) day: number for day of the year
4) week: day of the week as a character string
5) temp_2: max temperature 2 days prior
6) temp_1: max temperature 1 day prior
7) average: historical average max temperature
8) actual: max temperature measurement **(label)**
9) friend: your friend’s prediction, a random number between 20 below the average and 20 above the average

In [ ]:
import pandas as pd

In [ ]:
# Read in data as pandas dataframe and display first 5 rows
data = pd.read_csv('temps.csv')

In [ ]:
data.head(5)

In [ ]:
data.shape

# 2. Data Preparation

If we look at the dimensions of the data, we notice only there are only **348 rows, which doesn’t quite agree with the 365 days** we know there were in 2019. 

Looking through the data from the NCEP (National Centre for Environmental Prediction - https://www.ncep.noaa.gov/, **noticed several missing days**.

In this case, the **missing data will not have a large effect**, and the data quality is good because of the source from where it is collected. 

We also can see there are nine columns which represent eight features and the one target (‘actual’).

#### Preprocessing Steps

1) One-hot encoded categorical variables

2) Split data into features and labels

3) Convert to arrays

4) Split data into training and testing sets

In [ ]:
print('The shape of our dataset is:', data.shape)

### 2.1 Identify anomalies using summary statistics.

In [ ]:
# Descriptive statistics for each column
data.describe()

#### There are not any data points that immediately appear as anomalous and no zeros in any of the measurement columns.


#### Another method to verify the quality of the data is by making basic plots. Often it is easier to spot anomalies in a graph than in numbers.

In [ ]:
# Use datetime for dealing with dates
import datetime

In [ ]:
# Get years, months, and days
years = data['year']
months = data['month']
days = data['day']

# List and then convert to datetime object
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]
#dates

In [ ]:
dates

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt

%matplotlib inline

# Set the style
plt.style.use('fivethirtyeight')

In [ ]:
# Set up the plotting layout
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize = (10,10))
fig.autofmt_xdate(rotation = 45)

# Actual max temperature measurement
ax1.plot(dates, data['actual'])
ax1.set_xlabel(''); ax1.set_ylabel('Temperature'); ax1.set_title('Max Temp')

# Temperature from 1 day ago
ax2.plot(dates, data['temp_1'])
ax2.set_xlabel(''); ax2.set_ylabel('Temperature'); ax2.set_title('Previous Day Max Temp')

# Temperature from 2 days ago
ax3.plot(dates, data['temp_2'])
ax3.set_xlabel('Date'); ax3.set_ylabel('Temperature'); ax3.set_title('Two Days Prior Max Temp')

# Friend Estimate
ax4.plot(dates, data['friend'])
ax4.set_xlabel('Date'); ax4.set_ylabel('Temperature'); ax4.set_title('Friend Estimate')

plt.tight_layout(pad=2)

### 2.2 One-Hot Encoding

#### Takes categorical variables, such as days of the week and converts it to a numerical representation without an arbitrary ordering. 

- Days of the week are intuitive to us because we use them all the time. 

- You will (hopefully) never find anyone who doesn’t know that ‘Mon’ refers to the first day of the workweek, but machines do not have any intuitive knowledge. What computers know is numbers and for machine learning we must accommodate them. We could simply map days of the week to numbers 1–7, but this might lead to the algorithm placing more importance on Sunday because it has a higher numerical value.

One hot encoding takes this:

| week |
|------|
| Mon  |
| Tue  |
| Wed  |
| Thu  |
| Fri  |

and converts it into:

| Mon | Tue | Wed | Thu | Fri |
|-----|-----|-----|-----|-----|
| 1   | 0   | 0   | 0   | 0   |
| 0   | 1   | 0   | 0   | 0   |
| 0   | 0   | 1   | 0   | 0   |
| 0   | 0   | 0   | 1   | 0   |
| 0   | 0   | 0   | 0   | 1   |

In [ ]:
# One-hot encode categorical features
data = pd.get_dummies(data)
data.head(5)

In [ ]:
print('Shape of data after one-hot encoding:', data.shape)

### 2.3 Convert Features and Labels and Convert Data to Arrays

#### 1) Separate the data into features and targets. 

The target, also known as the label, is the value we want to predict, in this case the actual max temperature and the features are all the columns the model uses to make a prediction. 

#### 2) We will also convert the Pandas dataframes to Numpy arrays 

because that is the way the algorithm works. (save the column headers, which are the names of the features, to a list to use for later visualization).

In [ ]:
# Use numpy to convert to arrays
import numpy as np

# Labels are the values we want to predict
labels = np.array(data['actual'])

# Remove the labels from the features
# axis 1 refers to the columns
features= data.drop('actual', axis = 1)
# Convert to numpy array
features = np.array(features)

# Saving feature names for later use
feature_list = list(data.columns)

### 2.4 Split the data into Training and Testing Sets

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, 
                                                                            labels, 
                                                                            test_size = 0.25,
                                                                            random_state = 42)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

# 3. Simple Baseline Predictions - Just compute average

Before we can make and evaluate predictions, we need to establish a baseline, a sensible measure that we hope to beat with our model. 

If our model cannot improve upon the baseline, then it will be a failure and we should try a different model or admit that machine learning is not right for our problem. 

**The baseline prediction for our case can be the historical max temperature averages.**

In other words, our baseline is the error we would get if we simply predicted the average max temperature for all days.

In [ ]:
features

In [ ]:
feature_list.index

In [ ]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('average')]

# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2), 'degrees.')

### Just doing average gave a forecast with an error of 5.06 degrees...Now, let us target to build a model with error less than our base line which is 5.06 degrees

# 4. Train Model

We import the random forest regression model from skicit-learn, instantiate the model, and fit (scikit-learn’s name for training) the model on the training data. (setting the random state for reproducible results).

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Instantiate model - 100 trees
rf = RandomForestRegressor(n_estimators= 100, 
                           random_state=42)

In [ ]:
# Train the model on training data
rf.fit(train_features, train_labels)

# 5. Make Predictions on Test Data

Our model has now been trained to learn the relationships between the features and the targets. The next step is figuring out how good the model is! To do this we make predictions on the test features (the model is never allowed to see the test answers). 

We then compare the predictions to the known answers. When performing regression, we need to make sure to use the absolute error because we expect some of our answers to be low and some to be high. 

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

###  Model Error = 3.92 degrees < Base Line Error = 5.06 degrees

##### It is nearly 25% better than the baseline...Still we may not be happy

# 6. Determine Performance Metrics

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# 7. Improve Model - Tuning

We can create models with different hyperparameters to try and boost performance. The only way to find the best ones
are to try a few and evaluate them! 

In [ ]:
rf_new = RandomForestRegressor(n_estimators = 500, 
                               criterion = 'mse', 
                               max_depth = 5
                              )
# Train the model on training data
rf_new.fit(train_features, train_labels)

In [ ]:
# Use the forest's predict method on the test data with new model rf_new
predictions2 = rf_new.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions2 - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

#### New model has improved the error from 3.92 to 3.81
#### We can tune other hyper paramters as well...

# 8. Variable importance and feature selection

At this point, we know our model is good, but it’s pretty much a black box. We feed in some Numpy arrays for training, ask it to make a prediction, evaluate the predictions, and see that they are reasonable. 

The question is: how does this model arrive at the values? 

we can look at the feature importances of our explanatory variables.

### 8.1 Variable Importances

- In order to quantify the usefulness of all the variables in the entire random forest, we can look at the relative importances of the variables. 

- The importances returned in Skicit-learn represent how much including a particular variable improves the prediction.

In [ ]:
# simplest way..
rf.feature_importances_

In [ ]:
# Get numerical feature importances - Represent in a better way
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

In [ ]:
# Print out the feature and importances 
[print('Variable: {:15} Importance: {}'.format(*pair)) for pair in feature_importances];

- First, At the top of the list is temp_1, the max temperature of the day before. This tells us the best predictor of the max temperature for a day is the max temperature of the day before, a rather intuitive finding. 

- The second most important factor is the historical average max temperature, also not that surprising. 

- Your friend turns out to not be very helpful, along with the day of the week, the year, the month, and the temperature 2 days prior. 

- These importances all make sense as we would not expect the day of the week to be a predictor of maximum temperature as it has nothing to do with weather. 

- Moreover, the year is the same for all data points and hence provides us with no information for predicting the max temperature.

### 8.2 Model with Two Most Important Features

we can remove those variables that have no importance and the performance will not suffer. Additionally, if we are using a different model, say a support vector machine, we could use the random forest feature importances as a kind of feature selection method.

In [ ]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, 
                                          random_state=42)

# Extract the two most important features
important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]

# Train the random forest
rf_most_important.fit(train_important, train_labels)

In [ ]:
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)

errors = abs(predictions - test_labels)

# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape

print('Accuracy:', round(accuracy, 2), '%.')

This tells us that we actually do not need all the data we collected to make accurate predictions! If we were to continue using this model, we could only collect the two variables and achieve nearly the same performance. In a production setting, we would need to weigh the decrease in accuracy versus the extra time required to obtain more information. 

## 8.3 Visualizations

### 8.3.1 Variable Importances

In [ ]:
# list of x locations for plotting
x_values = list(range(len(importances)))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Axis labels and title
plt.ylabel('Importance')
plt.xlabel('Variable')
plt.title('Variable Importances')

### 8.3.2. Predictions and Actual Values

we can plot the entire dataset with predictions highlighted. This requires a little data manipulation, but its not too difficult. We can use this plot to determine if there are any outliers in either the data or our predictions.

In [ ]:
# Dates of training values
months = features[:, feature_list.index('month')]
days = features[:, feature_list.index('day')]
years = features[:, feature_list.index('year')]

# List and then convert to datetime object
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# Dataframe with true values and dates
true_data = pd.DataFrame(data = {'date': dates, 'actual': labels})

# Dates of predictions
months = test_features[:, feature_list.index('month')]
days = test_features[:, feature_list.index('day')]
years = test_features[:, feature_list.index('year')]

# Column of dates
test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]

# Convert to datetime objects
test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]

# Dataframe with predictions and dates
predictions_data = pd.DataFrame(data = {'date': test_dates, 'prediction': predictions}) 

In [ ]:
# Plot the actual values
plt.plot(true_data['date'], true_data['actual'], 'b-', label = 'actual')

# Plot the predicted values
plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'prediction')
plt.xticks(rotation = '60'); 
plt.legend()

# Graph labels
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual and Predicted Values');


### Targets and Data Visualization

The previous graph doesn’t look as if we have any noticeable outliers that need to be corrected. To further diagnose the model, we can plot residuals (the errors) to see if our model has a tendency to over-predict or under-predict, and we can also see if the residuals are normally distributed. However, I will just make one final chart showing the actual values, the temperature one day previous, the historical average, and our friend’s prediction. This will allow us to see the difference between useful variables and those that aren’t so helpful.

In [ ]:
# Make the data accessible for plotting
true_data['temp_1'] = features[:, feature_list.index('temp_1')]
true_data['average'] = features[:, feature_list.index('average')]
true_data['friend'] = features[:, feature_list.index('friend')]

# Plot all the data as lines
plt.plot(true_data['date'], true_data['actual'], 'b-', label  = 'actual', alpha = 1.0)
plt.plot(true_data['date'], true_data['temp_1'], 'y-', label  = 'temp_1', alpha = 1.0)
plt.plot(true_data['date'], true_data['average'], 'k-', label = 'average', alpha = 0.8)
plt.plot(true_data['date'], true_data['friend'], 'r-', label = 'friend', alpha = 0.3)

# Formatting plot
plt.legend(); plt.xticks(rotation = '60');

# Lables and title
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual Max Temp and Variables');

we can see why the max temperature one day prior and the historical max temperature are useful for predicting max temperature while our friend is not

## EXERCISE

- Experiments to Compute the trade-off - Accuracy vs Runtime - Which model takes more compute time for training and inference?
- Try other Algorithms such as SVM for regression and compare the results
- Hyper paramter tuning - Explore different methods and compare the results